# CLLM generation & curation tutorial

# Setup keys and experiment

In [1]:
%cd /content
!rm -rf CLLM
!git clone https://github.com/seedatnabeel/CLLM.git
%cd /content/CLLM
!ls -la

/content
Cloning into 'CLLM'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 166 (delta 21), reused 19 (delta 17), pack-reused 133 (from 1)
Receiving objects: 100% (166/166), 109.08 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (85/85), done.
Updating files: 100% (74/74), done.
/content/CLLM
total 448
drwxr-xr-x 9 root root   4096 Feb  8 19:36 .
drwxr-xr-x 1 root root   4096 Feb  8 19:35 ..
-rw-r--r-- 1 root root    115 Feb  8 19:36 AUTHORS.rst
drwxr-xr-x 2 root root   4096 Feb  8 19:36 data
drwxr-xr-x 8 root root   4096 Feb  8 19:36 .git
-rw-r--r-- 1 root root   3206 Feb  8 19:36 .gitignore
-rw-r--r-- 1 root root    401 Feb  8 19:36 .isort.cfg
-rw-r--r-- 1 root root  11357 Feb  8 19:36 LICENSE
drwxr-xr-x 4 root root   4096 Feb  8 19:36 notebooks
-rw-r--r-- 1 root root 358991 Feb  8 19:36 overview_cllm.png
-rw-r--r-- 1 root root   1320 Feb  8 19:36 .pre-commit-config.yaml
-rw-r--r-

In [2]:
!pip -q install -e /content/CLLM

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for CLLM (pyproject.toml) ... done


In [3]:
import cllm, sys, os
print("cllm imported from:", cllm.__file__)
print("Python:", sys.version)
print("CWD:", os.getcwd())


cllm imported from: /content/CLLM/src/cllm/__init__.py
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
CWD: /content/CLLM


In [4]:
!pip -q install -U openml liac-arff


In [5]:
import os, re

path = "/content/CLLM/src/cllm/llm_gen.py"
assert os.path.isfile(path), f"Not found: {path}"

with open(path, "r", encoding="utf-8") as f:
    s = f.read()

# Thay import cũ bằng try/except
pattern = r"from\s+langchain\.prompts\s+import\s+ChatPromptTemplate"
replacement = (
    "try:\n"
    "    from langchain.prompts import ChatPromptTemplate\n"
    "except Exception:\n"
    "    from langchain_core.prompts import ChatPromptTemplate\n"
)

if re.search(pattern, s):
    s2 = re.sub(pattern, replacement, s, count=1)
    with open(path, "w", encoding="utf-8") as f:
        f.write(s2)
    print("Patched:", path)
else:
    print("No matching import found. Showing lines containing 'ChatPromptTemplate':")
    for i, line in enumerate(s.splitlines(), 1):
        if "ChatPromptTemplate" in line:
            print(i, line)


Patched: /content/CLLM/src/cllm/llm_gen.py


In [6]:
from cllm.utils import *
from cllm.llm_gen import *
from cllm.data_loader import *

import pandas as pd
from copy import deepcopy
import time
from sklearn.model_selection import train_test_split

print("Imports OK")


Imports OK


In [7]:
import sys
sys.path.append('src/')

from cllm.utils import *
from cllm.llm_gen import *
from cllm.data_loader import *

import pandas as pd
from copy import deepcopy
import time
from sklearn.model_selection import train_test_split


#############################################################
# API KEY SETUP INSTRUCTIONS
#############################################################

# for vllm
#api_key = "EMPTY"
#api_base = "http://localhost:8000/v1"

# for together
# api_key = "add together api key"
# api_base = "https://api.together.xyz/v1"


# for azure openai
# api_key = "EMPTY"
# api_base = "add azure deployment link"

# for openai
# api_key = "EMPTY"
# api_base = DO NOT INCLUDE

#############################################################

api_details = {
    "api_base": "http://127.0.0.1:8000/v1",
    "api_version": None,
    "api_key": "EMPTY",
}


model_short_name = 'mixtral' # 'gpt-4' (do not use other short names)
model = "Qwen/Qwen2.5-1.5B-Instruct" # "gpt4_20230815" (use name of your model deployment)
llm_serving='vllm' # supported 'azure_openai', 'together', 'vllm'

seed = 0
ns = 20 # n_samples per class. e.g. if binary = 40 samples (i.e. 20 per class)
dataset = 'compas'
n_synthetic=10 # just to test --- normall should be 1000
n_processes = 5

# STEP 1: Generation

## Get dataset

In [8]:
df_feat, df_label, df = get_data(dataset=dataset, seed=seed)

X_train, X_remain, y_train, y_remain = sample_and_split(df_feat, df_label, ns=ns, seed=seed)

X_val, X_test, y_val, y_test = train_test_split(
    X_remain, y_remain, test_size=0.5, random_state=seed
)


X_train_orig = deepcopy(X_train)
y_train_orig = deepcopy(y_train)

/content/CLLM/src/cllm/data_loader.py:323: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code will continue to work. You can use the dataframe's `to_numpy` function to continue using numpy arrays.
  X, y, categorical_indicator, attribute_names = dataset.get_data(


## Setup Prompt

In [9]:
!pip -q uninstall -y langchain langchain-core langchain-community
!pip -q install "langchain==0.1.16" "langchain-core==0.1.45" "langchain-community==0.0.34"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.13.0.90 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.37.0 requires numpy>=2.0, but you have numpy 1.26.4 which is i

In [10]:
import json
import pandas as pd

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

example_df = pd.concat([X_train_orig, y_train_orig], axis=1)
example_df = example_df.sample(frac=1, random_state=42).reset_index(drop=True)

output_parser = JsonOutputParser()
format_instructions = output_parser.get_format_instructions()

generator_template = """\
You are a synthetic data generator.
Your goal is to produce data which mirrors the given examples in causal structure and feature and label distributions
but also produces as diverse samples as possible.

I will give you real examples first.

Leverage your knowledge about criminal recidivism to generate 50 realistic but diverse samples.

example data: {data}

{format_instructions}

Rules:
- Return a JSON array of objects.
- Each object must have exactly the same keys as the columns in the example data.
- Return only JSON. No markdown. No commentary.
- Do not wrap JSON in triple backticks.
- Do not copy the examples; generate new diverse samples.
"""

prompt = ChatPromptTemplate.from_template(generator_template)

few = example_df.head(30).to_dict(orient="records")

# Debug prompt text (CHỈ truyền đúng các placeholder có trong template)
prompt_text = prompt.format(
    data=json.dumps(few, ensure_ascii=False),
    format_instructions=format_instructions,
)

print(prompt_text[:1200])


Human: You are a synthetic data generator.
Your goal is to produce data which mirrors the given examples in causal structure and feature and label distributions
but also produces as diverse samples as possible.

I will give you real examples first.

Leverage your knowledge about criminal recidivism to generate 50 realistic but diverse samples.

example data: [{"sex": 0.0, "age": 28.0, "juv_fel_count": 0.0, "juv_misd_count": 0.0, "juv_other_count": 0.0, "priors_count": 1.0, "age_cat_25-45": 1.0, "age_cat_Greaterthan45": 0.0, "age_cat_Lessthan25": 0.0, "race_African-American": 0.0, "race_Caucasian": 1.0, "c_charge_degree_F": 0.0, "c_charge_degree_M": 1.0, "y": 0}, {"sex": 1.0, "age": 22.0, "juv_fel_count": 0.0, "juv_misd_count": 0.0, "juv_other_count": 0.0, "priors_count": 1.0, "age_cat_25-45": 0.0, "age_cat_Greaterthan45": 0.0, "age_cat_Lessthan25": 1.0, "race_African-American": 0.0, "race_Caucasian": 1.0, "c_charge_degree_F": 1.0, "c_charge_degree_M": 0.0, "y": 0}, {"sex": 1.0, "age": 

In [11]:
!pip -q install -U vllm openai

# kill nếu có server cũ
!pkill -f "vllm.entrypoints.openai.api_server" || true

# start server nền + ghi log
!nohup python -m vllm.entrypoints.openai.api_server \
  --model "Qwen/Qwen2.5-1.5B-Instruct" \
  --host 0.0.0.0 \
  --port 8000 \
  --dtype auto \
  --gpu-memory-utilization 0.85 \
  > vllm_server.log 2>&1 &

!echo "Started. Showing first lines of log:" && head -n 20 vllm_server.log


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 134.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [12]:
print(api_details)
print("llm_serving:", llm_serving)
print("model:", model)


{'api_base': 'http://127.0.0.1:8000/v1', 'api_version': None, 'api_key': 'EMPTY'}
llm_serving: vllm
model: Qwen/Qwen2.5-1.5B-Instruct


In [13]:
!lsof -i :8000 | head
!ps aux | grep -E "vllm|api_server" | head -n 20


root        3487  0.0  0.3 136044 42588 ?        D    19:41   0:00 python3 -m vllm.entrypoints.openai.api_server --model Qwen/Qwen2.5-1.5B-Instruct --host 0.0.0.0 --port 8000 --dtype auto --gpu-memory-utilization 0.85
root        3511  0.0  0.0   7376  3568 ?        S    19:41   0:00 /bin/bash -c ps aux | grep -E "vllm|api_server" | head -n 20
root        3513  0.0  0.0   6484  2580 ?        S    19:41   0:00 grep -E vllm|api_server


In [14]:
!ls -la | grep vllm
!tail -n 200 vllm_server.log


-rw-r--r-- 1 root root      0 Feb  8 19:41 vllm_server.log


In [15]:
!ss -ltnp | grep ":8000" || true

In [16]:
!pkill -f "vllm.entrypoints.openai.api_server" || true

!python -m vllm.entrypoints.openai.api_server \
  --model "Qwen/Qwen2.5-1.5B-Instruct" \
  --host 0.0.0.0 \
  --port 8000 \
  --dtype auto \
  --gpu-memory-utilization 0.80


^C
2026-02-08 19:41:59.600752: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770579719.799296    3542 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770579719.852226    3542 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770579720.229273    3542 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770579720.229317    3542 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770579720.229321    3542 computation_placer.cc:177] computation placer 

In [ ]:
!nvidia-smi

## Generate using LLM

In [ ]:
retries = 4  # Max retries you want to attempt

while retries > 0:
    try:

        if len(example_df)>20:
            ic_samples=20
        else:
            ic_samples=len(example_df)

        print(f'Running {dataset}, {seed}, {model} --- {n_processes}')
        df_llm = llm_gen(prompt, generator_template, format_instructions, example_df,
                        n_samples=n_synthetic,
                        temperature=0.9,
                        max_tokens=1000, model=model,
                        n_processes=n_processes,
                        ic_samples=ic_samples,
                        llm_serving=llm_serving,
                        api_details=api_details)


        break  # if successful, break out of the loop
    except Exception as e:
        time.sleep(120)
        print(f"Error: {e}. Retrying with reduced n_processes...")
        n_processes = int(n_processes/2)
        retries -= 1
        if n_processes < 1:
            print("Error: Minimum n_processes reached. Exiting...")
            break
# try:
tmp_df = df_llm.astype(example_df.dtypes)
df_llm = tmp_df
# except:
#     pass

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpx/_client.py", line 979, in _send_handling_redirects
    response = self._send_single_request(request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpx/_client.py", line 1014, in _send_single_request
    response = transport.handle_request(request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 249, in handle_request
    with map_httpcore_exceptions():
         ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/contextlib.py", line 158, in __exit__
    self.gen.throw(value)
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 118, in map_httpcore_exceptions
    raise mapped_exc(message) from exc
httpx.ConnectError: [Errno 111] Connection refused

The above excep

In [ ]:
df_llm

## Process LLM generated data to have the same data types

In [ ]:
df_llm = df_llm.dropna()
df_llm = df_llm[~df_llm.apply(lambda row: any([isinstance(cell, str) and cell in ['integer', 'float', 'numeric', 'categorical', 'number', 'No', 'Yes', 'continuous', 'age in years', 'string'] for cell in row]), axis=1)]

example_df = deepcopy(X_train_orig)
example_df['y'] = deepcopy(y_train_orig)

try:
    df_llm = df_llm.astype(example_df.dtypes)
except:
    # Assuming the dtypes from the example_df['Dtrain'].dataframe() is what you want
    target_dtypes = example_df.dtypes.to_dict()

    problematic_rows = set()

    for col, dtype in target_dtypes.items():
        for index, value in df[col].items():
            try:
                _ = dtype.type(value)  # Try to convert the value
            except Exception:
                problematic_rows.add(index)

    # Convert the problematic rows to a list and sort them
    problematic_rows = sorted(list(problematic_rows))

    # Drop the problematic rows
    df_llm.drop(problematic_rows, inplace=True)

    # Identify rows where any cell is of type list
    rows_with_lists = df.applymap(lambda x: isinstance(x, list)).any(axis=1)

    # Drop those rows
    df_llm = df_llm[~rows_with_lists]

    df_llm = df_llm.astype(example_df.dtypes)


df_llm

In [ ]:
df_llm.dtypes

# STEP 2: Curation

In [ ]:
from src.curation import data_centric_curation

X_check = df_llm.drop(columns=['y'])
y_check = df_llm['y'].values.astype(int)

curation_metric = 'aleatoric'
curation_ythresh=0.2
curation_xthresh=0 #adaptive

easy_train, ambig_train, unlearnable_train, Curator_xgb = data_centric_curation(X_train_orig, y_train_orig, X_check, y_check,
                 curation_metric=curation_metric, retrain=False, nest = 100,
                 curation_ythresh=curation_ythresh, curation_xthresh=curation_xthresh)

curated_train_ids = np.concatenate((easy_train, ambig_train))
curated_train_ids, unlearnable_train